# データ加工

《学修項目》
* ◎集計処理、四則演算処理
* ◎ソート処理、サンプリング処理
* ◎クレンジング処理（外れ値、異常値、欠損値）
* 結合処理（内部結合、外部結合）
* データ型変換処理
* データの標準化、ダミー変数

《キーワード》
> 量的データ，質的データ, 単純集計, クロス集計, ソート処理とマージ処理, 統合処理とクエリ, 内部結合と外部結合, データクレンジング，データクリーニング, 欠損値処理, 外れ値処理, 異常値処理, ノイズの除去, データの型変換, スケーリング（正規化，標準化）


《参考文献，参考書籍》
*   [1] [東京大学MIセンター公開教材 「2-5 データ加工」](http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf) [《利用条件CC BY-NC-SA》](https://creativecommons.org/licenses/by-nc-sa/4.0/deed.ja)
*   [2] [データサイエンスのためのデータベース（データサイエンス入門シリーズ）（講談社）](https://www.kspub.co.jp/book/detail/5193105.html)
*   [3] [数理・データサイエンス・AI公開講座（放送大学）](https://www.ouj.ac.jp/booklet/2022/29_2022_MDS-AI.pdf)

## 1. データの集計

あらまし
* データベースにおける用語
* データの種類（量的データ，質的データ）
* 単純集計
* クロス集計
* ソート処理とマージ処理


### 1.1 データ加工（data processing）とは

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=4"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_04.jpg' alt='データ加工とは' width='640' border='1'></a>
</figure>

### 1.2 データベースにおける用語

* テーブル（表），カラム（列），レコード（行）RDBではタプル，フィールド値（セル値）　※（カッコ）はExcelでの呼び方

#### Python pandasライブラリで行単位でデータを作ってみる

例）9011から9016までの6名の学生の性別とクラス名，3科目のテストの点数（整数で100点満点）のテーブル

In [52]:
## Python pandasライブラリで列単位でデータを作って，カラム・レコード・フィールド値単位で見てみる
## 参考:https://pystyle.info/pandas-table-styling-cheatsheet/
import numpy as np
import pandas as pd
from IPython.display import display, HTML

# CSVデータなどから read_csv して dataへ読み込んでも良い．
data = [["Male", "A", 84, 70, 96], ["Male", "B", 79, 79, 79], ["Female", "A", 89, 92, 86], 
        ["Male", "A", 99, 76, 98], ["Female", "C", 56, 60, 70], ["Female", "C", 46, 44, 47]]

df = pd.DataFrame(data,
    index=["9011", "9012", "9013", "9014", "9015", "9016"],
    columns=["gender", "class", "Math", "English", "Physics"],
)

# 各行の合計を新たな列 Row_Total として追加する（横軸方向で数値のみ）
df.loc[:,'Row_Total'] = df.sum(numeric_only=True, axis=1)

In [53]:
# English カラム（列） を緑色にする
css_bg_b = {"background-color": "green"}
display(df.style.set_properties(subset="English", **css_bg_b))

,gender,class,Math,English,Physics,Row_Total
9011,Male,A,84,70,96,250
9012,Male,B,79,79,79,237
9013,Female,A,89,92,86,267
9014,Male,A,99,76,98,273
9015,Female,C,56,60,70,186
9016,Female,C,46,44,47,137


In [54]:
# id "9013"番 の レコード（行） をオレンジ色にする
css_bg_o = {"background-color": "orange"}
display(df.style.set_properties(subset=pd.IndexSlice["9013", :], **css_bg_o))

,gender,class,Math,English,Physics,Row_Total
9011,Male,A,84,70,96,250
9012,Male,B,79,79,79,237
9013,Female,A,89,92,86,267
9014,Male,A,99,76,98,273
9015,Female,C,56,60,70,186
9016,Female,C,46,44,47,137


In [55]:
# 数値に応じたカラースケール（ヒートマップ）で表示する
df.style.background_gradient()

,gender,class,Math,English,Physics,Row_Total
9011,Male,A,84,70,96,250
9012,Male,B,79,79,79,237
9013,Female,A,89,92,86,267
9014,Male,A,99,76,98,273
9015,Female,C,56,60,70,186
9016,Female,C,46,44,47,137


#### データの種類

* 量的データ：数字など（四則演算できる） 例）各科目の「点数」（これの合計を求めることができた）
* 質的データ：文字列など（四則演算できない） 例）学生番号，性別，クラス名（識別子として，別々のものとして認識することができる）

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=13"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_13.jpg' alt='整然データ' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=14"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_14.jpg' alt='整然データの利点' width='640' border='1'></a>
</figure>

### 1.3 集計処理
* 単純集計
    * データ件数の数え上げ

* 算術演算，統計演算と組み合わせる
    * 四則演算や統計量
    * 条件付き集計

* クロス集計
    * 複数項目の比較

ソート処理とマージ処理
* データの並び替え
* データの統合


### 1.4 集計処理の例（単純集計）

集計（カウント）
* 単純な集計処理
    * データの件数を計算：項目ごとでデータ集計
    * コンピュータでの実装：ExcelやPython(pandas)

⇒ 一つの項目（属性）の性質や，ばらつきを見るのに適している

#### 例題：学生テーブルを作ってみる

* 入力：学生番号，性別，クラス名の一覧表
* 出力：「性別」「クラス」項目別で集計

In [56]:
# 学生番号 : [性別， クラス名] の一覧表を得る（列名マッチで抽出）
df[["gender", "class"]]

,gender,class
9011,Male,A
9012,Male,B
9013,Female,A
9014,Male,A
9015,Female,C
9016,Female,C


In [57]:
# [性別， クラス名] の一覧 から， 性別毎の度数を得る（gender で groupby.count() を使う）
df[["gender", "class"]].groupby('gender').count()

,class
gender,
Female,3
Male,3


In [58]:
# [性別， クラス名] の一覧 から， クラス毎の度数を得る（class で groupby.count() を使う） 
df[["gender", "class"]].groupby('class').count()

,gender
class,
A,3
B,1
C,2


### 1.5 集計処理の例（演算処理）

* 算術演算
    * 四則演算などを組み合わせる

* 度数分布表
    * データを階層別に集計する
    * ヒストグラムを作成し可視化する

* 基本統計量の集計
    * 最大値，最小値，平均値などを求める
    * 箱ひげ図を作成し可視化する

#### 例題：各科目の点数について，度数分布表を作ってみる

* 入力：各科目の点数
* 出力：Math, English, Physicsの度数分布

In [59]:
# 学生番号 : [Math， English, Physics] の一覧表を得る（列名マッチで抽出）
df[["Math", "English", "Physics"]]

,Math,English,Physics
9011,84,70,96
9012,79,79,79
9013,89,92,86
9014,99,76,98
9015,56,60,70
9016,46,44,47


In [60]:
# 科目名 列名マッチで抽出後， [60,100]を４区間に分割した度数分布を value_count で求める
df["Math"].value_counts(bins=np.linspace(60, 100, 5), sort=False)
## df["English"].value_counts(bins=np.linspace(60, 100, 5), sort=False)
## df["Physics"].value_counts(bins=np.linspace(60, 100, 5), sort=False)

(59.999, 70.0]    0
(70.0, 80.0]      1
(80.0, 90.0]      2
(90.0, 100.0]     1
Name: Math, dtype: int64

### 1.6 クロス集計

* 項目を2つ以上かけ合わせて集計する手法
    * 項目（属性）ごとの違いを見る
    * 分割表ともいう
    * 表側（目的変数）と表頭（説明変数）

例）アイスクリームの売上分析の場合，目的変数：アイスクリームの種類，説明変数：その日の気温や天気　などとなる．

* 統計量を組み合わせる
    * 数学の平均をクロス集計する：データの精度が問題になる
    * 各科目の平均をクロス集計する：科目毎の値のばらつきが問題になる

#### 例題：男女別，クラス別 で 各科目の平均点のクロス分析を行ってみる

* 入力：学生番号，性別，クラス名，各科目の平均点の一覧表
* 出力：男女別，クラス別 の 平均点（クロス分析）

In [61]:
# 3科目の点数の平均値を新たな列 Row_Average として追加する
df.loc[:,'Row_Average'] = df[["Math", "English", "Physics"]].sum(axis=1) / 3  # 3科目であることが分かっている

# インタラクティブ・テーブル機能を使って，ソートなど簡単な分析は可能だが，クロス分析は難しい
df

,gender,class,Math,English,Physics,Row_Total,Row_Average
9011,Male,A,84,70,96,250,83.333333
9012,Male,B,79,79,79,237,79.000000
9013,Female,A,89,92,86,267,89.000000
9014,Male,A,99,76,98,273,91.000000
9015,Female,C,56,60,70,186,62.000000
9016,Female,C,46,44,47,137,45.666667


In [62]:
# crosstabを使って， 性別 vs クラス のクロス集計（人数）を行う
pd.crosstab(index=df['gender'],columns=df['class'])

class,A,B,C
gender,,,
Female,1,0,2
Male,2,1,0


In [63]:
# crosstabを使って， 性別 vs クラス の 学生平均点のクロス集計（点数）を行う
pd.crosstab(index=df['gender'],columns=df['class'],values=df['Row_Average'],aggfunc='mean') 

# 該当する学生が居ないフィールドは，欠損値(NaN)として処置されている【重要】

class,A,B,C
gender,,,
Female,89.000000,NaN,53.833333
Male,87.166667,79.0,NaN


In [ ]:
# 科目別の点数の平均値を新たな列 Column_Average として追加する
df.loc[:,'Column_Average'] = df[["Math", "English", "Physics"]].sum(axis=0) / 3  # 3科目であることが分かっている

# インタラクティブ・テーブル機能を使って，ソートなど簡単な分析は可能だが，クロス分析は難しい
df

In [ ]:
# crosstabを使って， [[性別],[クラス]] vs 3科目平均点のクロス集計（点数）を行う
pd.crosstab(index=df['gender'],columns=df['class'],values=df['Row_Average'],aggfunc='mean') 

### 1.7 データの種類

構造化データ
* 分析できる形に構造化されたデータ
* 表形式で整理されたデータセット
→量的データ，質的データに分類される

非構造化データ
* まだ構造化されていないデータ
→ 画像，音声，文章データなど　※データ表現の章を参照

### 1.8 データのソート

* 順序付けの基準（オーダ）
    * 量的変数の場合：科目の得点など（間隔尺度（例：西暦年など），数値尺度（例：重量など）の別がある）
    * 質的変数の場合：名義尺度（学生名など），順序尺度（学籍番号など．順序が存在し，数量によるソートが可能）

昇順と逆順
* 数値の大小で順序付ける：質的変数も，順序尺度であれば可能
* 名義尺度も あいうえお順，ABC順などで順序付け (aab < ac < adbc など文字列にもオーダがつけられる）

### 1.9 ソート処理の例

大きさを指標として並び替える
* 数量データや順序データ
* 文字列のソート：漢字データは，ひらがな　などを付与してオーダをつける（読み方の違いは漢字コード列の情報だけでは不充分）

※ふりがなが付いていないと，データ重複，名寄せ処理の妨げになる【重要】
- きっかわさん？よしかわさん？
- おやまさん？こやまさん？

### 1.10 名寄せ

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=15"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_15.jpg' alt='データ加工とは' width='640' border='1'></a>
</figure>

## 2. データの統合処理

あらまし
* 統合処理とクエリ
* データの結合処理
* 内部結合と外部結合

### 2.1 データ統合とは？

統合処理とクエリ
* クエリ＝データベース管理システムに対する問い合わせ，要求
    * SQLで実装する
* プログラムではデータを検索し，変換・整形して出力する．SQLクエリを内部の実装として含むこともある
    * 統合処理の条件指定などを行う必要がある

### 2.2 データの結合処理

* 複数のファイルやデータを（何らかの基準，キーで）結合する処理
* データベースでは，内部結合と外部結合の別がある
* ExcelやPython(pandas)で実装

### 2.3 データベースの結合例

* 複数のデータ集合の結合
    * 学籍簿と成績表の結合など
    * 2つのデータの集合を結ぶ「キー」が必要

* データベースの結合処理
    * 内部結合と外部結合の別，外部キーが必要
    * Union, Join といった処理が提供されている．

* 一般的なデータの結合処理
    * Excelの場合：データツール > 統合
    * Python pandasライブラリの場合：join

### 2.4 内部結合と外部結合

データベース（表）の内部結合
* 結合条件に指定している値が両方のテーブルに存在するデータを抽出する結合
* 2つのテーブル間で，共通（結合条件の項目の値が一致）するレコードのみを抽出

データベース（表）の外部結合
* 基準となるテーブルに存在すれば抽出する結合．どちらのテーブルを基準にするかを指定し，その基準となるテーブルに存在するデータを抽出，基準ではないテーブルからは抽出できるデータのみ取得する．
* 左外部結合（Left Outer Join），右外部結合（Right Outer Join）の別がある
* 完全外部結合は，全ての要素を入れる

参考：https://ai-inter1.com/pandas-dataframe_join/

### 2.5 データベース（表）の内部結合の例

* 両方のテーブルに存在するデータを抽出して結合する
* 2つのテーブル間で，結合条件のフィールド値が一致するレコードのみを出力とする

### 2.6 データベース（表）の外部結合の例

* 一方のテーブルについて全レコードを抽出し，もう一方のテーブルについて結合条件のフィールド値と一致するデータのみ抽出
* 全抽出するテーブルの位置により，左外部結合（Left Outer Join），右外部結合（Right Outer Join）が存在する
* 完全外部結合は，全ての要素を入れる

## 3. データの前処理とデータ変換

### 3.1 データの前処理

データクレンジング，データクリーニング
* 破損したデータ，不正確なデータ，無関係のデータなどの処理
* 住所や名簿などをデータ分析可能な状態にすること

主なデータ前処理
* 欠損値処理
* 外れ値処理，異常値処理
* ノイズの除去

データの変換
* 前処理としてのデータの変換
* 型変換やスケーリング
* 正規化，標準化

### 3.2 欠損値処理

データクレンジング，データクリーニング
* 欠損値の前処理：欠けたピースを埋める．どんな値で埋めるのか？
    * 一般的には，数値データは平均値で置き換える．質的データは最頻値で置き換える（検討が必要）

ノイズの除去
* センサ値などでノイズが乗った形で出現する
* 正常値と置き換えるなどの処理が考えられる（検討が必要）
    * 例）売上の時系列データの場合，ノイズが生じた場合には過去の同じような時期のデータと比較するなど

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=17"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_17.jpg' alt='データクレンジング' width='640' border='1'></a>
</figure>

### 3.3 欠損値の扱い

欠損値を処理する一般的な方法

* 削除：列や行ごと削除する
* 平均置換：平均値で置き換える（当該レコードの信頼性が毀損する）
* 最頻値で置き換え：質的データに有効
* ダミー置換：ダミー値（Unknown, nan, Zeroなど）で置き換える

完全データ：全ての値が観測されているデータ
不完全データ：欠損が含まれているデータ

頻繁に欠損値が出現するが，量的データのモデルが既知の場合には，欠損データを予測可能な回帰モデルを作成し，逐次代入する方法がとられる（統計的数値補完）

参考：https://qiita.com/Ryota7101/items/6c0a539ddf08a39a9868

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=19"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_19.jpg' alt='欠損値への対処' width='640' border='1'></a>
</figure>

### 3.4 外れ値処理，異常値処理

* 外れ値：通常とはかけ離れた値
    * 異常の大きい，小さいなど
* 外れ値処理
    * 一般的には「残す」

+異常値：存在し得ない値など（例：非常に小規模な図書館の入場者が100万人であることを示すデータ，など）
    * 欠損値と同じ扱いとする
    * 個別に吟味し，置き換えや削除を検討する

一般的に，データ件数，レコード数が非常に大きくなるど，欠損値・外れ値・異常値を含む可能性が高まる

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=18"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_18.jpg' alt='外れ値の検定' width='640' border='1'></a>
</figure>

### 3.5 前処理：データ変換

データ変換処理の必要性
* 生データは「汚れている」「扱いにくい」
* データの型変換やスケーリングといった前処理が必要

データの型変換
* データ型：プログラミング言語における型変換，整数型，小数点型など
* データサイエンスにおけるデータ変換（スケーリング：正規化，標準化）

プログラミング言語によっては，変数データの型変換コマンドが実装されている

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=16"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_16.jpg' alt='データ型変換' width='640' border='1'></a>
</figure>

### 3.6 データ変換処理

データサイエンスにおけるデータ変換
* 量的データの変換
* 質的データから量的データへの変換（オーダ，ラベリング）

量的データの変換
* スケーリング（尺度）の変更，対数化など
* 正規化，標準化

質的データの変換
* 質的データから量的データへの変換（順序尺度：例 成績が1番，2番, 3番 であった場合，1番の方が3番より成績が良いことは分かるが，1番と3番を「足した」結果は意味をもたない）
* 量的データへの割当て（ラベリング，名義尺度：例 良い=1, 普通=2, 悪い=3）

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=21"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_21.jpg' alt='ダミー変数' width='640' border='1'></a>
</figure>

### 3.7 スケーリング：正規化，標準化

複数のデータを比較するためにデータの形をそろえる
* それぞれの指標に基づいてスケーリングする（尺度を統一する）
* 平均，標準偏差，最大，最小といった統計値を利用する

標準化：平均と標準偏差を利用する
* Z-score 標準化法
* 平均0, 分散1でスケーリング

正規化：最大と最小を利用する
* Min-Max 正規化法
* 最小を0, 最大を1でスケーリング

参考：https://aiacademy.jp/media/?p=1147

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=20"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_20.jpg' alt='データの正規化・標準化' width='640' border='1'></a>
</figure>

### 3.8 データの標準化と正規化の例

標準化の例
1. 数学の点数を標準化
2. 平均値と標準偏差を求める
3. 平均からの差を標準偏差で割る

正規化の例
1. 最大値と最小値の差を求める
2. 最小値からの差を求める
3. 正規化値を求める（1.で割る）

参考：https://aiacademy.jp/media/?p=1147

## 4. Python pandasライブラリによるデータ加工・分析の例

TBA

## 5. (オプション) Excel Power Query によるデータ加工・分析[1]

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=5"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_05.jpg' alt='Excel Power Query' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=6"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_06.jpg' alt='Excel Power Query' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=7"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_07.jpg' alt='Excel Power Query' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=8"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_08.jpg' alt='Excel Power Query' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=9"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_09.jpg' alt='Excel Power Query' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=10"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_10.jpg' alt='Excel Power Query' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=11"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_11.jpg' alt='Excel Power Query' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=12"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_12.jpg' alt='Excel Power Query' width='640' border='1'></a>
</figure>

## 6. (参考) ビッグデータ処理ツール

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-5_data_processing.pdf#page=22"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/5/UTAI-text/JPEG/2-5_data_processing_22.jpg' alt='データ加工とは' width='640' border='1'></a>
</figure>

* [Apache Spark](https://spark.apache.org/)
* [Apache Hadoop](https://hadoop.apache.org/)
* [HillView](https://github.com/vmware/hillview)


# memo

## XX3. リレーショナルデータベース (RDB)とSQLによる操作

あらまし

* SQL (Structured Query Language)
* クエリ（Query:検索）
* 関係データベースに対する演算


### 3.1 リレーショナルデータベース(RDB)とは

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=5"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_05.jpg' alt='RDBとは' width='640' border='1'></a>
</figure>

### 3.2 データベース管理言語 (DML) 

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=6"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_06.jpg' alt='DMLとは' width='640' border='1'></a>
</figure>

### 3.3 SQL (Structured Query Language)
関係データベース言語
+ スキーマ定義やデータ操作などを行うための言語
+ ISO (国際標準機構) によって標準化
+ 異なる関係データベース管理システム上で動作（プットフォーム非依存．ミドルウェアと呼ぶ）
+ 関係代数の演算（制限，射影，結合，直積）を記述する

### クエリ（Query:検索）
+ 検索結果を得るための問い合わせ
+ リレーショナルデータベースでは，データの登録，検索，削除，更新をSQLによって行うことができる．

### 3.4 関係データベースに対する演算
+ 制限：いくつかのタプル（行）の抽出
+ 射影：いくつかの属性（カラム）の抽出
+ 結合：ある属性で関係AとBを結びつける
+ 直積：関係AとBの組み合わせの「全てのパターン」を得る

#### 3.4.1 制限
ある条件に該当するタプル（行）の取得
表計算ソフトの場合，必要が行だけを残す（選択）することに相当する

> 例）家賃がX万円以上，Y万円以下のアパートを探したい<br>
> 駅から徒歩T分以内のホテルを探したい
など

```
# SQLによる操作例
SELECT * FROM purchase_history
WHERE ID = '001001'
```


#### 3.4.2 射影
ある属性（カラム）だけを取得
表計算ソフトの場合，必要が列だけを残す（選択）することに相当する

> 例）賃貸DBから，家賃と駅からの距離の2カラムだけに絞りたい<br>
> カメラの購入時比較のため，名前，値段，画素数だけに絞りたい
> など

```
# SQLによる操作例
SELECT ID, Name, Item 
FROM purchase_history
```

#### 3.4.3 直積
複数の関係（テーブル）を単純に直積集合で求める

> 例）顧客テーブル，レンタル履歴テーブル，レンタル商品を結合して，ある人が過去に借りた商品を（横断的に）調べたい

※表計算ソフトでは非常に高コストな操作となるが，RDBであればviewという形で実現できる

```
# SQLによる操作例
SELECT ID, faculties.Name, Age, courses.Name
FROM faculties, courses
```


#### 3.4.4 結合
複数の関係（テーブル）の直積集合のviewから，必要なレコードだけを抽出する
結合 := 直積 >> 制限 とも言える

```
# SQLによる操作例
SELECT customers.ID, Name, Age, Item
FROM customers, purchase_history
WHERE customers.ID = purchase_history.ID
```



### 3.5 SQL言語によるデータベース操作の実例

#### 3.5.1 気象データベースからの検索例 (Python SQLite3)

In [64]:
# CSVファイルをpandasで読み込んでデータフレームdfに格納
import sys
import pandas as pd
from IPython.display import display
import sqlite3

df = pd.read_csv('JMA_Nagano_2021.csv')
df.columns = ['date','temp_ave','temp_high','temp_low','rain_total','sun_shine','wind_ave','wind_max','wind_dir','wind_peak','wind_dir2','wind_dir16','humid_ave','humid_max','humid_ave2','forecast_day','forecast_night']

##display(df)

FileNotFoundError: ignored

In [ ]:

# データベースを新規作成して、接続する
dbname = ('weather.db') #データベース名.db拡張子で設定（カレントディレクトリに通常のファイルとして実体がある）
conn = sqlite3.connect(dbname, isolation_level=None) #データベースを作成、自動コミット機能ON
cursor = conn.cursor() #カーソルオブジェクトを作成


In [ ]:
# nagano2021テーブル　： 年月日,平均気温(℃),最高気温(℃),最低気温(℃),降水量の合計(mm),日照時間(時間),平均風速(m/s),最大風速(m/s),風向,最大瞬間風速(m/s),風向,最多風向(16方位),平均蒸気圧(hPa),最小相対湿度(％),平均湿度(％),天気概況(昼：06時〜18時),天気概況(夜：18時〜翌日06時)
sql = """CREATE TABLE IF NOT EXISTS nagano2021(date,temp_ave,temp_high,temp_low,rain_total,sun_shine,wind_ave,wind_max,wind_dir,wind_peak,wind_dir2,wind_dir16,humid_ave,humid_max,humid_ave2,forecast_day,forecast_night)"""

cursor.execute(sql) #executeコマンドでSQL文を実行
conn.commit() #データベースにコミット
cursor = conn.cursor() #カーソルオブジェクトを作成

In [ ]:
# pandas DataFrameから nagano2021テーブルに全てインポートする
df.to_sql('nagano2021',conn,if_exists='append',index=None)
conn.commit() #データベースにコミット

In [ ]:
## クエリ例1) nagano2021テーブルの temp_ave が 29度から35度の間であった日を抽出する
sql = """SELECT * FROM nagano2021 WHERE temp_ave BETWEEN 29 and 35;"""
cursor.execute(sql)
print(cursor.fetchall())#全レコードを取り出す

In [ ]:
## クエリ例2) nagano2021テーブルの 天気概況(日中) forecast_day に "みぞれ" が含まれる日を抽出する
sql = """SELECT * FROM nagano2021 WHERE forecast_day LIKE '%みぞれ%';"""
cursor.execute(sql)
print(cursor.fetchall())#全レコードを取り出す

In [ ]:
## クエリ例3) nagano2021テーブルの 3月中で， 天気概況 forecast_day に "雪" が含まれる日を抽出する
sql = """SELECT * FROM nagano2021 WHERE date BETWEEN '2021/3/1' and '2021/3/31' AND forecast_day LIKE '%雪%';"""
cursor.execute(sql)
print(cursor.fetchall())#全レコードを取り出す

#### 3.5.2 ERモデルから関係データベースへの変換例 (Python SQLite3)

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=13"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_13.jpg' alt='外部キー' width='640' border='1'></a>
</figure>

データベース course, 親テーブル subject, department, 子テーブル registrationを作成して，ERモデルに基づいた関係データベースを作成してみる（外部キー参照による制約付き）

In [ ]:
## ERモデルから関係データベースへの変換例 (SQLite3)
import sqlite3

# データベースを新規作成して、接続する
dbname = ('course.db') #データベース名.db拡張子で設定（カレントディレクトリに通常のファイルとして実体がある）
conn = sqlite3.connect(dbname, isolation_level=None) #データベースを作成、自動コミット機能ON
cursor = conn.cursor() #カーソルオブジェクトを作成

In [ ]:
# subject [科目]テーブルを作成する（主キー：id）
sql = """CREATE TABLE IF NOT EXISTS subject(id varchar(4) primary key, name varchar(32))"""

cursor.execute(sql) #executeコマンドでSQL文を実行
conn.commit() #データベースにコミット

In [ ]:
# 作成したテーブルにレコードを複数行格納する
sql = """INSERT INTO subject VALUES(?,?)"""

data = [
   ("1001", "統計学"),
   ("1002", "データベース")
]
cursor.executemany(sql, data)#SQL文を実行

In [ ]:
# department [部局]テーブルを作成する（主キー：id）
sql = """CREATE TABLE IF NOT EXISTS department(id varchar(2) primary key, name varchar(32))"""

cursor.execute(sql) #SQL文を実行

In [ ]:
# 作成したテーブルにレコードを複数行格納する
sql = """INSERT INTO department VALUES(?,?)"""

data = [
   ("03", "理学部"),
   ("05", "薬学部"),
   ("07", "法学部")
]
cursor.executemany(sql, data)#SQL文を実行

In [ ]:
sql = """SELECT * FROM subject"""
cursor.execute(sql)
print(cursor.fetchall())#全レコードを取り出す
sql = """SELECT * FROM department"""
cursor.execute(sql)
print(cursor.fetchall())#全レコードを取り出す

In [ ]:
# registration [開講科目]テーブルを作成する（外部キー：subject_id, department_id）
sql = """CREATE TABLE IF NOT EXISTS registration(id varchar(4) primary key, subject_id varchar(4), department_id varchar(2), foreign key(subject_id) references subject(id), foreign key(department_id) references department(id))"""
cursor.execute(sql) #SQL文を実行

sql = """PRAGMA foreign_keys=true; """ # SQLite3 このテーブルについて外部キー参照を有効化する
cursor.execute(sql) #SQL文を実行


In [ ]:
# 作成したテーブルにレコードを複数行格納する
sql = """INSERT INTO registration VALUES(?,?,?)"""

data = [
   ####("0201", "1999", "03"), # 外部キー設定してあるので，存在しない科目番号を指定するとエラーになる
   ("0301", "1001", "03"),
   ("0302", "1002", "03"),
   ("0501", "1001", "05")
]
cursor.executemany(sql, data)#SQL文を実行

In [ ]:
sql = """SELECT * FROM registration"""
cursor.execute(sql)
print(cursor.fetchall())#全レコードを取り出す

#### SQLの代表的なコマンド（リファレンス）

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=11"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_11.jpg' alt='SQL言語による操作' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=8"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_08.jpg' alt='SQL言語による操作' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=9"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_09.jpg' alt='SQL言語による操作' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=10"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_10.jpg' alt='SQL言語による操作' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=7"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_07.jpg' alt='SQL言語による操作' width='640' border='1'></a>
</figure>

### 3.6 RDB以外のデータベース

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=18"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_18.jpg' alt='ERモデル' width='640' border='1'></a>
</figure>

## XX2. 主キーと外部キー

あらまし

* 関係モデル
* スキーマ
* 関係モデルの定式化
* 関係従属性とキー
* 関数従属性
* 主キー, 外部キー，参照制約

### 2.1 関係モデルとしてのリレーショナルデータベース(RDB)

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=5"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_05.jpg' alt='RDBとは' width='640' border='1'></a>
</figure>

### 関係モデル

データを関係で表現するモデル．RDBとは関係表の集合と言える

+ データテーブル：実体
+ データ項目：属性集合（スキーマ）
+ フィールド名：属性（カラム）
+ 各データレコード：タプル（行）

→ 上記全てをあわせて　関係　と呼ぶ

### 2.2 スキーマ

表の名前と属性の集合 ＜表の名前＞（＜属性1＞，…）のように表現する

> 例）学生（学生番号, 名前, 入学年）<br>
> 科目（科目番号，科目名，中間試験の有無）

### 2.3 関係モデルの定式化の例

+ 属性：科目（科目番号，科目名，中間試験の有無）
+ 属性の定義域：科目番号={C001,C002,...}，科目名={データ管理, 情報基礎, ...}，中間試験の有無={有,無}

+ 関係 R ⊆ 科目番号 x 科目名 x 中間試験の有無　（直積集合）

→ 各データレコードは，関係R の 元となる．

### 2.4 関係従属性とキー
関係内のルールのようなもの
+ 関係従属性の例：「学生番号が決まれば，名前も一意に決まる」
+ キーの例：「学生番号が決まれば，1人の学生を特定できる」

誰が関係従属性を決めるのか？
+ データの性質によって決めていく
+ RDB設計者の裁量による

### 2.5 関数従属性の例

【定義】ある関係Rに，関係従属性 X→Y がある<br>
　　　⇔関係Rにおいて，属性Xの値が属性Yの値を一意に決定する

つまり，任意の2つのタプルの属性Xの値が同じであれば，それらのタプルYの値も同じである．

> 例）学生番号→名前，科目番号→科目名が関数従属性を有する

### 2.6 主キー, 外部キー，参照制約 【重要】

主キー：データベースの設計者が属性集合の中から１つ選んだキーのこと
+ スキーマの属性に下線を引くことで，主キーを表す

> 例）学生（<u>学生番号</u>, 名前, 入学年）<br>
> 科目（<u>科目番号</u>，科目名，中間試験の有無）
※主キーで，1つのタプル（データレコード）を指定することが多い

外部キー：関係表の表のある列に，別の関係表の特定の列に登録されている項目した入力できないように指定する属性のこと

参照制約：外部キーを設定することで，参照先の関係表の指定された列に，登録された値しか書き込むことができない制約のこと（複数の実体集合をデータ項目とした関係（子テーブル）で用いられる）

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=12"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_12.jpg' alt='主キー' width='640' border='1'></a>
</figure>

<figure>
<a href="http://www.mi.u-tokyo.ac.jp/pdf/2-4_data_base.pdf#page=13"><img src='https://raw.githubusercontent.com/MDASH-shinshu/MDASH-T-DE/main/4/UTAI-text/JPEG/2-4_data_base_13.jpg' alt='外部キー' width='640' border='1'></a>
</figure>